## Chargement package et données 

In [168]:
import numpy as np
import pandas as pd
import os
import math
os.getcwd()

file_in = '../data/Resultats_formulaires_21_octobre.csv'
file_out = 'data_CROUS/preprocessed_hippo/formulaire_hippo_preprocessed.csv'

In [169]:
df1 = pd.read_csv(file_in, sep=";")
# csv_2 = pd.read_csv("Resultats_formulaires_22_octobre.csv", sep=";")
# csv_3 = pd.read_csv("Resultats_formulaires_28_octobre.csv", sep=";")
# csv_4 = pd.read_csv("Resultats_formulaires_29_octobre.csv", sep=";")
# csv_tot = pd.concat([csv_1, csv_2,csv_3, csv_4])
# csv_tot = csv_tot.set_index("N° Formulaire")

## Reduction du CSV aux données seules (sans les annotations en bas)

In [170]:
#on garde les lignes remplies des tableurs
df1 = df1.iloc[:135, :]


## Nettoyage des colonnes Amis 

In [171]:
#on créé une regex qui va nous permettre de récupérer les codes des formations
import re 
#on peut aussi le faire avec cdette syntaxte :
# bool(re.match(regex, 'hello1'))

def KeepOnlyForm(x):
    regex = r"[0-9][A-Z][0-9][A-Z]"
    if type(x)== str:
        match = re.search(regex,x)
        if match != None:
            return match.string
        else:
            return pd.NA
    else:
        return pd.NA

In [172]:
# on selectionne les colonnes qui commencent par "Ami" (on garde maximum 5 amis)
col_amis = df1.columns.str.startswith("Ami")
dfamis = df1.loc[:,col_amis]

#toutes les valeurs qui ne sont pas des chaines de caractères sont remplacées par des Sandofour
dfamis = dfamis.applymap(KeepOnlyForm)



In [173]:

#onremplace les colonnes par les colonnes nettoyées
df1.loc[:,col_amis] = dfamis
#on enleve les colonnes après la colonne "Ami 5"
df1 = df1.iloc[:,:df1.columns.get_loc("Ami 5")+1]


In [174]:
#ce genre d'espace qui traine après ami 3...
#df1 = df1.drop(columns=["Ami 1", "Ami 2","Ami 3 ","Ami 4","Ami 5",'Unnamed: 28','Unnamed: 27','Unnamed: 29'])

## Variables catégorielles binaires sont codées en 0 et 1

In [ ]:
'''
one hot sur une variable à deux états ça sert à rien si ? 
'''
# temp = pd.get_dummies(df1["Genre"])
# df1.drop(columns=["Genre"], inplace=True)
# df1 = pd.concat([df1, temp], axis=1)

# temp = pd.get_dummies(df1["Végé"])
# df1.drop(columns=["Végé"], inplace=True)
# df1 = pd.concat([df1, temp], axis=1)

# temp = pd.get_dummies(df1["Régime particulier"])
# df1.drop(columns=["Régime particulier"], inplace=True)
# df1 = pd.concat([df1, temp], axis=1)


In [176]:
#on remplace les valeurs de genre par des 0 et des 1
df1["Genre"] = df1["Genre"].replace({"Femme":0, "Homme":1})
#on remplace les valeurs de végétarien par des 0 et des 1
df1["Végé"] = df1["Végé"].replace({"Oui":1, "Non":0})
#on remplace les valeurs de régime particulier par des 0 et des 1
df1["Régime particulier"] = df1["Régime particulier"].replace({"Oui":1, "Non":0})
#on remplace les valeurs de proximité par des 0 et des 1
df1["Proximité"] = df1["Proximité"].replace({"Oui":1, "Non":0})


## Suppression des colonnes inutiles 

In [177]:
#on supprime la colonne fréquence Crous
df1.drop(columns=["Fréquence CROUS"], inplace=True)

In [178]:
df1.drop(columns=["Seule ou en groupe"], inplace=True)

In [179]:
df1.drop(columns=["Connaissance"], inplace=True)

In [180]:
df1.drop(columns=["Nb file"], inplace=True)

In [181]:
df1.drop(columns=["Parti groupe"], inplace=True)

## Les variables catégorielles qui présente une relation d'odre sont traitées comme  des variables numériques

In [ ]:
#on recupere les valeurs possibles de la colonne stress
stress = df1["Stress"].unique()
print(stress)

In [183]:
#on remplace les valeurs de la colonne Stress par des valeurs numériques
df1["Stress"] = df1["Stress"].replace("Non", 0)
df1["Stress"] = df1["Stress"].replace("Plutôt non", 1)
df1["Stress"] = df1["Stress"].replace("Plutôt oui", 2)
df1["Stress"] = df1["Stress"].replace("Oui, très", 3)
df1["Stress"] = df1["Stress"].replace("N.A", 1.5) #on remplace les valeurs manquantes par la moyenne ??
#on normalise par moyenne et variance la colonne stress
df1["Stress"] = (df1["Stress"] - df1["Stress"].mean())/df1["Stress"].std()

In [184]:
#on recupere les valeurs possibles de la colonne stress
faim = df1["Faim"].unique()
print(faim)

['Oui,assez faim' 'Oui, très faim' 'Oui, assez faim faim'
 'Oui, mais très peu' 'Oui,très faim' 'Non' 'Oui, assez faim'
 'Oui, assez faim ' 'N.A']


In [185]:
#on remplace les valeurs de la colonne Faim par des valeurs numériques
df1["Faim"] = df1["Faim"].replace('Non', 0)
df1["Faim"] = df1["Faim"].replace('Oui, mais très peu', 1)
df1["Faim"] = df1["Faim"].replace("Oui,assez faim", 2)
df1["Faim"] = df1["Faim"].replace('Oui, assez faim faim', 2)
df1["Faim"] = df1["Faim"].replace('Oui, assez faim', 2)
df1["Faim"] = df1["Faim"].replace('Oui, assez faim ', 2)
df1["Faim"] = df1["Faim"].replace('Oui,très faim', 3)
df1["Faim"] = df1["Faim"].replace('Oui, très faim', 3)
df1["Faim"] = df1["Faim"].replace('N.A', 1.5) #on remplace les valeurs manquantes par la moyenne ??
#on normalise par moyenne et variance la colonne faim 
df1["Faim"] = (df1["Faim"] - df1["Faim"].mean())/df1["Faim"].std()

In [186]:
#on recupere les valeurs possibles de la colonne Activité physique
activite = df1["Activité physique"].unique()
print(activite)

['Importante' 'Faible' 'Modérée' 'N.A' 'Aucune' 'Aucune ']


In [187]:
#on remplace les valeurs de la colonne Activité physique par des valeurs numériques
df1["Activité physique"] = df1["Activité physique"].replace('Aucune', 0)
df1["Activité physique"] = df1["Activité physique"].replace('Aucune ', 0)
df1["Activité physique"] = df1["Activité physique"].replace("Faible", 1)
df1["Activité physique"] = df1["Activité physique"].replace('Modérée', 2)
df1["Activité physique"] = df1["Activité physique"].replace('Importante', 3)
df1["Activité physique"] = df1["Activité physique"].replace("N.A", 1.5) #on remplace les valeurs manquantes par la moyenne ??
#on normalise par moyenne et variance la colonne Activité physique 
df1["Activité physique"] = (df1["Activité physique"] - df1["Activité physique"].mean())/df1["Activité physique"].std()

In [ ]:
#on recupere les valeurs possibles de la colonne Activité physique
activite = df1["Percept senso"].unique()
print(activite)

In [ ]:
#on remplace les valeurs de la colonne Activité physique par des valeurs numériques
df1["Activité physique"] = df1["Activité physique"].replace('Aucune', 0)
df1["Activité physique"] = df1["Activité physique"].replace('Aucune ', 0)
df1["Activité physique"] = df1["Activité physique"].replace("Faible", 1)
df1["Activité physique"] = df1["Activité physique"].replace('Modérée', 2)
df1["Activité physique"] = df1["Activité physique"].replace('Importante', 3)
df1["Activité physique"] = df1["Activité physique"].replace("N.A", 1.5) #on remplace les valeurs manquantes par la moyenne ??
#on normalise par moyenne et variance la colonne Activité physique 
df1["Activité physique"] = (df1["Activité physique"] - df1["Activité physique"].mean())/df1["Activité physique"].std()

In [ ]:
#on recupere les valeurs possibles de la colonne Activité physique
activite = df1["Influ sociale"].unique()
print(activite)

In [ ]:
#on remplace les valeurs de la colonne Activité physique par des valeurs numériques
df1["Activité physique"] = df1["Activité physique"].replace('Aucune', 0)
df1["Activité physique"] = df1["Activité physique"].replace('Aucune ', 0)
df1["Activité physique"] = df1["Activité physique"].replace("Faible", 1)
df1["Activité physique"] = df1["Activité physique"].replace('Modérée', 2)
df1["Activité physique"] = df1["Activité physique"].replace('Importante', 3)
df1["Activité physique"] = df1["Activité physique"].replace("N.A", 1.5) #on remplace les valeurs manquantes par la moyenne ??
#on normalise par moyenne et variance la colonne Activité physique 
df1["Activité physique"] = (df1["Activité physique"] - df1["Activité physique"].mean())/df1["Activité physique"].std()

In [ ]:
#on recupere les valeurs possibles de la colonne Activité physique
activite = df1["Equilibre alim"].unique()
print(activite)

In [ ]:
#on remplace les valeurs de la colonne Activité physique par des valeurs numériques
df1["Activité physique"] = df1["Activité physique"].replace('Aucune', 0)
df1["Activité physique"] = df1["Activité physique"].replace('Aucune ', 0)
df1["Activité physique"] = df1["Activité physique"].replace("Faible", 1)
df1["Activité physique"] = df1["Activité physique"].replace('Modérée', 2)
df1["Activité physique"] = df1["Activité physique"].replace('Importante', 3)
df1["Activité physique"] = df1["Activité physique"].replace("N.A", 1.5) #on remplace les valeurs manquantes par la moyenne ??
#on normalise par moyenne et variance la colonne Activité physique 
df1["Activité physique"] = (df1["Activité physique"] - df1["Activité physique"].mean())/df1["Activité physique"].std()

## Sauvegarde du formulaire 

In [188]:
df1.to_csv("formulaire_21_octobre_clean.csv", encoding='latin1')

## Sauvegarde d'un fichier concatenant formulaire et choxi pour chaque sujet

In [100]:
#concatenation du formulaire avec les choix 
choix1 = pd.read_csv("choix_21_octobre_clean.csv", encoding='latin1', sep=";") 
formulaire1 = pd.read_csv("formulaire_21_octobre_clean.csv", encoding='latin1')


In [102]:
#on associe choix et formulaire par le numero de formulaire
choix1 = choix1.set_index("num_form")
formulaire1 = formulaire1.set_index("N° Formulaire")
#on merge les deux dataframes
final1 = pd.merge(formulaire1, choix1, left_index=True, right_index=True)


In [ ]:
final1.to_csv("final_21_octobre_clean.csv", encoding='latin1')